In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [ ]:
anime_df = pd.read_csv('/content/drive/MyDrive/ML AND DL/Datasets/anime.csv')
rating_df = pd.read_csv('/content/drive/MyDrive/ML AND DL/Datasets/rating.csv')

In [ ]:
anime_ratings = pd.merge(rating_df,anime_df[['anime_id','name']], on='anime_id')

In [ ]:
user_ratings = anime_ratings.groupby('user_id')['rating'].count()
active_users = user_ratings[user_ratings >= 100].index
anime_ratings = anime_ratings[anime_ratings['user_id'].isin(active_users)]

anime_matrix = anime_ratings.pivot_table(index = 'user_id', columns = 'name', values = 'rating').fillna(0)

In [ ]:
knn_model = NearestNeighbors(metric='cosine',algorithm='brute')
knn_model.fit(anime_matrix.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
def get_recommendations(name):
  anime_idx = anime_matrix.columns.get_loc(name)
  distances, indices = knn_model.kneighbors(anime_matrix.iloc[:,anime_idx].values.reshape(1, -1), n_neighbors=6)

  recommended_anime = []
  for i in range(1, len(distances.flatten())):
    recommended_anime.append(
        {
            'Anime': anime_matrix.columns[indices.flatten()[i]],
            'Similarity': distances.flatten()[i]
        }
    )

  return recommended_anime